In [119]:
from PIL import Image, ImageDraw

# Define the mapping from integers to (image_id, rotation, flip)
tile_mapping = {
    0: ('Empty', 0, False),
    1: ('Curve', 0, False),
    2: ('Curve', 1, False),
    3: ('Curve', 2, False),
    4: ('Curve', 3, False),
    5: ('Straight', 0, False),
    6: ('Straight', 1, False),
    7: ('T turn', 0, False),
    8: ('T turn', 1, False),
    9: ('T turn', 2, False),
    10: ('T turn', 3, False),
    11: ('T turn', 0, True),
    12: ('T turn', 1, True),
    13: ('T turn', 2, True),
    14: ('T turn', 3, True),
}

def assemble_map_with_grid_and_cart(map_data, image_paths, cart_positions, cart_image_path):
    # Load images
    images = {key: Image.open(path) for key, path in image_paths.items()}
    cart_image = Image.open(cart_image_path)
    cart_width, cart_height = cart_image.size
    
    # Get map dimensions
    rows = len(map_data)
    cols = len(map_data[0])

    # Create a new blank image with the right size
    map_image = Image.new('RGBA', (cols * 100, rows * 100))

    for r in range(rows):
        for c in range(cols):
            tile_code = map_data[r][c]
            img_id, rotation, flip = tile_mapping[tile_code]
            img = images[img_id]
            
            # Apply rotation
            img = img.rotate(rotation * 90, expand=True)
            
            # Apply flip if needed
            if flip:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)
            
            # Paste the image into the map
            map_image.paste(img, (c * 100, r * 100))

    # Draw grid lines
    draw = ImageDraw.Draw(map_image)
    for r in range(rows + 1):
        draw.line([(0, r * 100), (cols * 100, r * 100)], fill='black', width=2)
    for c in range(cols + 1):
        draw.line([(c * 100, 0), (c * 100, rows * 100)], fill='black', width=2)

    # Add carts to the map
    for position in cart_positions:
        cart_row, cart_col, cart_rotation = position
        rotated_cart_image = cart_image.rotate(cart_rotation * 90, expand=True)
        rotated_cart_width, rotated_cart_height = rotated_cart_image.size
        x_offset = (100 - rotated_cart_width) // 2
        y_offset = (100 - rotated_cart_height) // 2
        map_image.paste(rotated_cart_image, (cart_col * 100 + x_offset, cart_row * 100 + y_offset), rotated_cart_image)

    return map_image

In [123]:
# Example usage
map_data = [
    [1, 8, 4],
    [11, 10, 13],
    [2, 6, 3]
]

cart_positions = [
    (1, 1, 3),  # (row, col, rotation in multiples of 90 degrees)
    (0, 2, 1)   # (row, col, rotation in multiples of 90 degrees)
]

image_paths  = {
    'Empty': 'images/Empty.png',
    'Curve': 'images/Curve.png',
    'Straight': 'images/Straight.png',
    'T turn': 'images/T turn.png',
}

cart_image_path = 'images/cart.png'

map_image = assemble_map_with_grid_and_cart(map_data, image_paths, cart_positions, cart_image_path)
map_image.show()  # This will display the assembled map
map_image.save('output_map_with_cart.png')  # This will save the assembled map to a file
